<a href="https://colab.research.google.com/github/CSenthusiast01/Ultra-Fast-Lane-Detection-v2/blob/master/ultrafast_lane_detection_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ultra Fast Lane Detection with Hybrid Anchor Driven Ordinal Classification.**

Note: add the path in /configs/tusimple_res18.py for the data_path and the log_path


create log file and the tusimpleroot file as mentioned in the repository.

In [1]:
# # Original Repository
# !git clone https://github.com/cfzd/Ultra-Fast-Lane-Detection-v2.git

# Updated code repository
!git clone https://github.com/CSenthusiast01/Ultra-Fast-Lane-Detection-v2.git

Cloning into 'Ultra-Fast-Lane-Detection-v2'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 164 (delta 41), reused 12 (delta 9), pack-reused 87
Receiving objects: 100% (164/164), 24.30 MiB | 25.31 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [2]:
%mkdir logs

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
ROOT = "/content/drive"
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/Ultra-Fast-Lane-Detection-v2
# %mkdir '$TUSIMPLEROOT'

/content/Ultra-Fast-Lane-Detection-v2


In [6]:
#for having dataset in the repository
# make sure to create the $TUSIMPLE directory in the project folder

# check the below command to copy the data from drive to the repo.
# %cp -r /content/drive/MyDrive/TuSimple_xsml/train_set/ /content/Ultra-Fast-Lane-Detection/'$TUSIMPLEROOT'/

%cp -r /content/drive/MyDrive/TuSimple_xsml/train_set/ /content/Ultra-Fast-Lane-Detection-v2/
!mv /content/Ultra-Fast-Lane-Detection-v2/train_set/ /content/Ultra-Fast-Lane-Detection-v2/'$TUSIMPLEROOT'/



In [7]:
# copying test_label.json in teh $TUSIMPLEROOT
%cp /content/drive/MyDrive/TuSimple_xsml/test_label.json /content/Ultra-Fast-Lane-Detection-v2/'$TUSIMPLEROOT'/

In [8]:
%cp /content/drive/MyDrive/TuSimple_xsml/test_set/test_tasks_0627.json /content/Ultra-Fast-Lane-Detection-v2/'$TUSIMPLEROOT'/

In [9]:
!python scripts/convert_tusimple.py --root '$TUSIMPLEROOT'
# this will generate segmentations and two list files: train_gt.txt and test.txt

  0% 0/3626 [00:00<?, ?it/s]/content/Ultra-Fast-Lane-Detection-v2/scripts/convert_tusimple.py:93: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all_points = np.zeros((4,56,2), dtype=np.float)
100% 3626/3626 [00:05<00:00, 690.89it/s]


## The below code cell is used to filter the train_gt.txt file


In [10]:
#EXPERIMENTAL SCRIPT TO FILTER DATA:

# %cd /content/Ultra-Fast-Lane-Detection/
# import os
# import numpy as np

# # The directory where you want to get the folder names
# directory1 = "/'$TUSIMPLEROOT'/clips/0313-1"
# directory2 = "/'$TUSIMPLEROOT'/clips/0313-2"
# directory3 = "/'$TUSIMPLEROOT'/clips/0531"
# directory4 = "/'$TUSIMPLEROOT'/clips/0601"

# # List all the files and folders in the directory
# entries1 = os.listdir(directory1)
# entries2 = os.listdir(directory2)
# entries3 = os.listdir(directory3)
# entries4 = os.listdir(directory4)

# # Filter out the ones that are not folders
# searcharr_0313_1 = [entry for entry in entries1 if os.path.isdir(os.path.join(directory1, entry))]
# searcharr_0313_1 = [entry for entry in entries2 if os.path.isdir(os.path.join(directory2, entry))]
# searcharr_0531 = [entry for entry in entries3 if os.path.isdir(os.path.join(directory3, entry))]
# searcharr_0601 = [entry for entry in entries1 if os.path.isdir(os.path.join(directory4, entry))]

# print(searcharr_0313_1)

# # Convert the list of folder names to a numpy array of strings
# array = np.array(folders)

# # Print the array
# print(array)

searcharr_0313_1=['0313-1/120', '0313-1/180','0313-1/240','0313-1/300','0313-1/360','0313-1/420','0313-1/480','0313-1/540','0313-1/60','0313-1/600']
searcharr_0313_2=['0313-2/20', '0313-2/25', '0313-2/30', '0313-2/35', '0313-2/40','0313-2/45', '0313-2/50', '0313-2/55', '0313-2/60', '0313-2/65']

searcharr_0531=['0531/1492626270684175793','0531/1492626274615008344','0531/1492626287507231547','0531/1492626327183261694','0531/1492626366226158339','0531/1492626371522989086','0531/1492626395742137012','0531/1492626487919028170','0531/1492626551883775328','0531/1492626688464435547']
searcharr_0601=['0601/1494452385593783358','0601/1494452391590105691','0601/1494452397586667021','0601/1494452403585050201','0601/1494452409581290246','0601/1494452415578787519','0601/1494452421576126921','0601/1494452427576086954','0601/1494452433570893263','0601/1494452439568910656']

clip_list = [searcharr_0313_1, searcharr_0313_2, searcharr_0531, searcharr_0601]
#update this loop to filter the contents from the train_gt.txt

mini_clip_list = [searcharr_0313_1] #micro dataset

filtered_data = []

#change to clip_list if want to use the original dataset or keep it as mini_clip_list if want micro dataset
for search_arr in clip_list:
  with open("$TUSIMPLEROOT/train_gt.txt", "r") as f:
    for line in f:
      # print("clip_name", clip_name)
      for i in range(0, len(search_arr)):
        if search_arr[i]+'/20.png' in line:
          filtered_data = filtered_data + [line]
with open("$TUSIMPLEROOT/train_gt.txt", "w") as f:
    f.seek(0) # to move to the start of the file
    for clip_data in filtered_data:
      f.write(clip_data)
    f.truncate()
print(len(filtered_data))




40


In [11]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 4.3 MB/s eta 0:00:00


In [12]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110
# Install Nvidia DALI (Very fast data loading lib))

%cd my_interp

!sh build.sh
# If this fails, you might need to upgrade your GCC to v7.3.0

Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.2/501.2 MB 2.0 MB/s eta 0:00:00
/content/Ultra-Fast-Lane-Detection-v2/my_interp
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ***********

In [13]:
# copying the testing data in the $TUSIMPLEROOT/clips (NOTE: ONLY DO THIS IF YOU WANT TO TEST THE DATA)


#THIS IS NOT WORKING AS INTENDED BECAUSE THERE IS ALREADY 0601 and 0531 in both the training as well as testing set.

%cp -r /content/drive/MyDrive/TuSimple_xsml/test_set/clips/* /content/Ultra-Fast-Lane-Detection-v2/'$TUSIMPLEROOT'/clips

In [21]:
#FILTERING THE text.txt FILE:
%cd /content/Ultra-Fast-Lane-Detection-v2/

searcharr_0530=['0530/1492626047222176976_0', '0530/1492626126171818168_0','0530/1492626127172745520_0','0530/1492626153155598528_0','0530/1492626155156451704_0','0530/1492626158152981904_0','0530/1492626166147797438_0','0530/1492626171146236124_0','0530/1492626191132352208_0','0530/1492626199127566374_0']
searcharr_0531=['0531/1492626253262712112','0531/1492626271917313999','0531/1492626272918083058','0531/1492626292371547028','0531/1492626323818607492','0531/1492626326050823425','0531/1492626328182638628','0531/1492626344839162069','0531/1492626364327255872','0531/1492626388446057821']
searcharr_0601=['0601/1494452381594376146','0601/1494452383592719171','0601/1494452387590928848','0601/1494452389590241875','0601/1494452393588330446','0601/1494452395587533775','0601/1494452399585761304','0601/1494452401584814468','0601/1494452405583118167','0601/1494452407582274573']

clip_list2 = [searcharr_0530, searcharr_0531, searcharr_0601]
#update this loop to filter the contents from the test.txt


mini_clip_list2 = [searcharr_0531] #micro dataset

filtered_data2 = []

#change to clip_list if want to use the original dataset or keep it as mini_clip_list if want micro dataset
for search_arr in clip_list2:
  with open("$TUSIMPLEROOT/test.txt", "r") as f:
    for line in f:
      # print("clip_name", clip_name)
      for i in range(0, len(search_arr)):
        if search_arr[i]+'/20.jpg' in line:
          filtered_data2 = filtered_data2 + [line]
          # print(filtered_data2)
with open("$TUSIMPLEROOT/test.txt", "w") as f:
    f.seek(0) # to move to the start of the file
    for clip_data in filtered_data2:
      f.write(clip_data)
    f.truncate()
print(len(filtered_data2))


/content/Ultra-Fast-Lane-Detection-v2
30


## **Before running the below train.py.  make sure to update the data_root and log_path in tusimple.py from the configs**

In [22]:
%cd /content/Ultra-Fast-Lane-Detection-v2/

/content/Ultra-Fast-Lane-Detection-v2


In [35]:
#before running this make sure to update the data_root an d log_path in tusimple.py from the configs
!python train.py configs/tusimple_res18.py

2024-01-28 11:50:21.982783: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 11:50:21.982837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 11:50:21.984264: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 11:50:21.991875: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 11:50:23.405740: W tensorflow/comp

## The trained weights are inside the logs folder as model_best.pth

## code for testing

In [42]:
#data visualization for trained weights - this creates test.avi video file for demo

#python demo.py configs/tusimple.py --test_model path_to_tusimple_18.pth

!python demo.py configs/tusimple_res18.py --test_model /content/logs/'20240128_115027_lr_1e-04_b_32'/model_best.pth

2024-01-28 12:00:05.280148: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 12:00:05.280199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 12:00:05.282099: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 12:00:05.295960: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 12:00:06.960183: W tensorflow/comp

In [43]:
# to display the video demo
from IPython.display import HTML
from base64 import b64encode
import os
# Compress video
os.system(f"ffmpeg -i /content/Ultra-Fast-Lane-Detection-v2/test.avi -vcodec libx264 video.mp4")
# Show video
mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
  <source src="%s" type="video/mp4">
</video>
""" % data_url)


In [ ]:
# Testing runtime

!python speed_simple.py

2024-01-27 13:46:39.083905: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-27 13:46:39.083952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-27 13:46:39.085313: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-27 13:46:39.092331: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-27 13:46:40.178954: W tensorflow/compiler/tf2

## below cell is optional dont run unnecessarily. This is for filtering the test_label.json if it contains more data than the test.txt file. Tjis code can also be used for filtering the training json files if used accordingly

In [ ]:
# filtering the test_label.json file (OPTIONAL) ------------ONLY RUN IF YOU DON'T HAVE PROPER test_label.json file

# Import the json module
import json

# Open the test.txt file and read the clip names
with open("$TUSIMPLEROOT/test.txt") as f:
  clip_names = f.read().splitlines()

# Open the test_label.json file and load the data
with open("$TUSIMPLEROOT/test_label.json") as f:
  data = [json.loads(line) for line in f]

# Create an empty list to store the filtered data
filtered_data = []

# Loop through the data and check if the clip name matches any of the clip names in the test.txt file
for item in data:
  if item["raw_file"] in clip_names:
    # If yes, append the item to the filtered data list
    filtered_data.append(item)

# Open a new file to write the filtered data
with open("$TUSIMPLEROOT/filtered_test_label.json", "w") as f:
  # Dump the filtered data list as json
  json.dump(filtered_data, f)

#rename the filtered_test_label.json as test_label.json and rename the original test_label.json to other name in order to run below cell

In [41]:
%mkdir /content/tmp


!python test.py configs/tusimple_res18.py --test_model /content/logs/'20240128_115027_lr_1e-04_b_32'/model_best.pth --test_work_dir /content/tmp


mkdir: cannot create directory ‘/content/tmp’: File exists
2024-01-28 11:59:25.794809: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 11:59:25.794858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 11:59:25.796158: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 11:59:25.805237: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate co

# deployment


In [ ]:
!python deploy/pt2onnx.py --config_path configs/tusimple_res18.py --model_path /content/logs/''/model_best.pth

In [ ]:
!trtexec --onnx=weights/culane_res34.onnx --saveEngine=/content/logs/tusimple_res18.engine

In [ ]:
!python deploy/trt_infer.py --config_path  configs/culane_res34.py --engine_path /content/logs/tusimple_res18.engine --video_path example.mp4

**## TO DO:**


1.   stop logging of each checkpoint or just manage the space correctly ✅
2.   add the test clips folders inside $TUSIMPLEROOT/clips folder ✅
3. update the test.txt to have only data about the available clips similar to how i did with the train_gt.txt file ✅
4. visualize the data as you did in the cascaded lane detection. ✅
5. A way to just take an random test image and see the segmentation as well as the classes just you did in the cascaded CNN lane detection
6. pretrained weights evaluation
7. find metrics for trained weights and biases ✅
8. Deployment and inference on tensorrt


# Training res32 model


In [ ]:
!python train.py configs/tusimple_res32.py